In [10]:
from selenium import webdriver

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='/Users/hoon/Library/Fonts/NanumSquareRegular.ttf').get_name()
plt.rc("font", family=font_name)
import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.request import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests

from tqdm import tqdm_notebook
import time

import re

# 문제 1.

+ https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=제주도날씨

In [ ]:
# 다음과 같은 결과물이 나오는 프로그램 만들기. def?
# 내가 데이터를 쓰기 위해
# 타인에게 데이터 제공을 서비스하기 위해 (인공지능 스피커)

# ----> 서울날씨:30 ℃ 	 구름많음, 어제보다 1˚ 높아요
# ----> 부산날씨:22 ℃ 	 흐림, 어제보다 4˚ 낮아요
# ----> 제주도날씨:22 ℃ 	 구름많음, 어제보다 0˚ 높아요
# ----> 광주날씨:27 ℃ 	 구름많음, 어제보다 1˚ 낮아요
# ----> 대전날씨:29 ℃ 	 구름많음, 어제보다 1˚ 낮아요
# ----> 경기도날씨:29 ℃ 	 구름많음, 어제보다 2˚ 낮아요

In [37]:
# "서울", "부산", "제주도", "광주", "대전", "경기도"

def weather_func(x):

    headers = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
    url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query={}날씨".format(x)

    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.text, 'html.parser')

    #온도
    temp_tmp = soup.find("div", class_="temperature_text").find("strong")
    temp = re.findall("[0-9]+", str(temp_tmp))[0]

    #날씨 상태
    condition = soup.find("div", class_="temperature_info")
    weather = condition.find("span", "weather before_slash").text

    # 비교군
    compare_tmp = condition.text
    compare = compare_tmp.split('  ')[0]

    print("----> {0}날씨:{1} ℃ 	 {2},{3}".format(x, temp, weather, compare))


weather_func("서울")

----> 서울날씨:7 ℃ 	 맑음, 어제보다 0° 높아요


# 문제 2.

+ http://ncov.mohw.go.kr/

In [ ]:
# --> 오늘의 신규 확진자 :전일대비 (+ 399)
# --> 현재까지 확진자 :(누적)148,273

In [86]:
def covid_today():
    headers = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
    url = "http://ncov.mohw.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun="
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')


    # 신규확진자
    base = soup.find_all('div','data_table mgt16 tbl_scrl_m mini')
    base = base[-1]
    new_tmp = base.find_all('tr')
    today = int(new_tmp[-2].find_all('td')[-2].text.replace(',',''))
    yesterday = int(new_tmp[-2].find_all('td')[-3].text.replace(',',''))

    headers = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
    url = "http://ncov.mohw.go.kr/"
    page = urlopen(url)
    # r = requests.get(url, headers=headers)
    # requests.get으로 접속하면 html, css가 안뜸.......?
    soup = BeautifulSoup(page, 'html.parser')

    base = soup.find_all("div", class_ = "box")
    sum = re.findall('[0-9]+', base[1].find("li", class_="person").text.replace(',',''))[0]
    
    
    print("""
    오늘의 신규확진자 : {0} 전일 대비({1})
    현재까지 확진자 : (누적){2}
    """.format(today, today - yesterday, sum))

covid_today()


    오늘의 신규확진자 : 2368 전일 대비(-152)
    현재까지 확진자 : (누적)39845393
    


# 문제 3.

+ www.naver.com

In [ ]:
# ---> 정은경 "백신 오접종 총 105건 발생…90건이 접종대상 오류"
# ---> G7 회의서 드러난 일본 속내…가입 반대·회담 일방취소
# ---> 코스피 일주일 만에 최고치 경신… 3,252.13 마감
# ---> 가해자 조사 뭉갠 공군검찰 수사 압색 후 '잠잠'…봐주기식 우려
# ---> 전두환 재판 불출석…조영대 신부 "부정해도 죄 가릴 수 없다"
# ---> 우체국 택배노조 "사회적 합의 어겨"…점거농성 돌입
# ---> 이준석 "파격이 새로움 넘어 여의도의 새 표준 돼야"
# ---> 박용진 "파죽지세로 나가겠다…우파 정책도 끌어안아야"
# ---> 청양 저수지 쇠줄 두른 시신은 50대 지역 주민
# ---> 머스크 비트코인 결제 중단 번복하자 비트코인 11% 뛰어

In [103]:
def today_news():
    url = "https://www.naver.com/"
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')

    news_tmp = soup.find("div", class_="list_issue")
    news = news_tmp.find_all("a", class_ = "issue")

    for i in range(len(news)):
        print("---> {0}".format(news[i].text))

today_news()

---> 1주일만에 또 병상확보 행정명령…수도권 준중병상 총454개 확보
---> 李 '동맹 그늘론' 역사관 논란 재연…野 "반미·운동권식 궤변"
---> 尹 "文정부, 한일관계 국내정치에 끌어들여 외교실종"
---> 행안위 소위, '민주당 방역지원금 예산 증액' 합의 불발
---> 제주공항 진에어 항공편 차츰 정상화…"오늘내로 최대한 수송"
---> 유류세 인하 첫날 휘발유 1천700원대로 하락…직영-알뜰주유소 북적
---> '도이치모터스 주가조작' 권오수 회장 영장…16일 구속심사
---> 문정인 "中올림픽서 4자 종전선언 지지시 차기정부 못뒤집어"
---> 구본준·신동빈…요소수 대란 속 '구원투수'로 나선 기업 CEO들
---> 이스타항공, 회생안 인가로 '재이륙'…AOC 발급·자금 확보 남아


# 문제 4.

+ https://search.naver.com/search.naver?sm=tab_hty.top&whttps://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%9D%8C%EC%9B%90%EC%B0%A8%ED%8A%B8&oquery=%EB%A9%9C%EB%A1%A0%EC%B0%A8%ED%8A%B8&tqi=UrZ0HsprvN8ssK5ZP%2BsssssstVh-314088here=nexearch&query=%EC%9D%8C%EC%9B%90%EC%B0%A8%ED%8A%B8&oquery=%EC%9D%8C%EC%9B%90&tqi=hKk9udp0J14ssfLos3Kssssssl4-396598

In [157]:
from selenium import webdriver

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='/Users/hoon/Library/Fonts/NanumSquareRegular.ttf').get_name()
plt.rc("font", family=font_name)
import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.request import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests

from tqdm import tqdm_notebook
import time

import re

def music_top10():
    url ="https://search.naver.com/search.naver?sm=tab_hty.top&whttps://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%9D%8C%EC%9B%90%EC%B0%A8%ED%8A%B8&oquery=%EB%A9%9C%EB%A1%A0%EC%B0%A8%ED%8A%B8&tqi=UrZ0HsprvN8ssK5ZP%2BsssssstVh-314088here=nexearch&query=%EC%9D%8C%EC%9B%90%EC%B0%A8%ED%8A%B8&oquery=%EC%9D%8C%EC%9B%90&tqi=hKk9udp0J14ssfLos3Kssssssl4-396598"
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    first_tmp = soup.find("ol", class_="music_list")
    second_tmp = soup.find("ol", class_="music_list _more_content")
    first_count = first_tmp.find_all("li", "list_item _sap_item _svp_item")
    second_count = second_tmp.find_all("li", "list_item _sap_item _svp_item")

    for i in range(len(first_count)):
        first_title = first_tmp.find_all("span", "tit_area")[i].text
        first_singer_album = first_tmp.find_all("div", "dsc_area")[i].find_all("span", "name")
        first_singer = first_singer_album[1].text
        first_album = first_singer_album[0].text

        print("{0}. {1} | {2} | {3}".format(i+1, first_title, first_singer, first_album))

    for i in range(len(second_count)):
        second_title = second_tmp.find_all("span", "tit_area")[i].text
        second_singer_album = second_tmp.find_all("div", "dsc_area")[i].find_all("span", "name")
        second_singer = second_singer_album[1].text
        second_album = second_singer_album[0].text

        print("{0}. {1} | {2} | {3}".format(i+6, second_title, second_singer, second_album))

music_top10()

1.  strawberry moon  | 아이유(IU) | strawberry moon
2.  Wake Up (Prod. 코드 쿤스트)  | 개코, Ourealgoat(아우릴고트), SINCE, 안병웅, Tabber, 조광일 | 쇼미더머니 10 Episode 1
3.  Savage  | aespa | Savage - The 1st Mini Album
4.  쉬어 (Feat. MINO) (Prod. GRAY)  | 아넌딜라이트(Anandelight), unofficialboyy, BE'O(비오), 지구인, Mudd the student | 쇼미더머니 10 Episode 1
5.  Stay  | The Kid LAROI, Justin Bieber | Stay
6.  신호등  | 이무진 | 신호등
7.  Next Level  | aespa | Next Level
8.  낙하 (with 아이유)  | AKMU(악뮤) | NEXT EPISODE
9.  My Universe  | Coldplay, 방탄소년단 | My Universe
10.  Bad Habits  | Ed Sheeran | =
